In [ ]:
# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#!pip install gdown
#!gdown --id 1lhyqkIn63bohxV7l4599zgD7R93TCkyM

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import sys
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
%matplotlib inline 


import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import layers
from keras import callbacks
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import shutil
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

train_size = 1000
total_train_size = 5000
filas_test = 1000
s = 300

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_train = pd.read_csv('/kaggle/input/medical-masks-part7/df_part_7.csv')
data_train = data_train.sample(frac=1)
data_train.head()

In [ ]:
data_train[:1000].groupby('TYPE').count()

In [ ]:
def load_imgs(start, stop):
    X_train = []
    y_train = data_train['TYPE'][start:stop]-1
    dirname = '/kaggle/input/medical-masks-part7/images'
    
    filenames = data_train[start:stop].name
    for filename in tqdm(filenames):
        image = cv2.imread(dirname + "/" + filename, cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_array = cv2.resize(image, (s,s))
        X_train.append(list(image_array))
    
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train = X_train / 255
    y_train = to_categorical(y_train, num_classes=4)

    X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=.35, shuffle=True)
    
    return [X_train, X_val, y_train, y_val]

# Model Definition

In [ ]:
# Define a simple sequential model
from keras.applications.xception import Xception
model=Xception(include_top = False, weights = 'imagenet', input_shape=(s,s,3))
flattened = tf.keras.layers.Flatten()(model.output)

fc1 = tf.keras.layers.Dense(4, activation='softmax', name="AddedDense2")(flattened)

model = tf.keras.models.Model(inputs=model.input, outputs=fc1) 

# Display the model's architecture
model.summary()

tf.keras.utils.plot_model(
     model,
     to_file="/kaggle/working/model.png",
     show_shapes=True,
     show_layer_names=True,
     rankdir="TB",
     expand_nested=True,
     dpi=96,
 )

In [ ]:
cd /kaggle/working

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping   
callbacks_list = [  
    ModelCheckpoint('/kaggle/working/AMD.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True),
]  

In [ ]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Model Training

In [ ]:
X_train, X_val, y_train, y_val = load_imgs(0, 1000)

history = model.fit(X_train, y_train, epochs=30, verbose=1, validation_data=(X_val,y_val), callbacks=[tensorboard_callback])

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
model.save('/kaggle/working/trained_model.h5')

# Test Prediction

In [ ]:
y_test = data_train['TYPE'][-filas_test:]-1
y_test = np.array(y_test)

In [ ]:
X_test = []
y_test = data_train['TYPE'][-filas_test:]-1
dirname = '/kaggle/input/medical-masks-part7/images'
    
filenames = data_train[-filas_test:].name
for filename in tqdm(filenames):
    imframe = cv2.imread(dirname + "/" + filename)
    imframe = cv2.cvtColor(imframe, cv2.COLOR_BGR2RGB)
    imframe = cv2.resize(imframe, (s,s))
    X_test.append(imframe)

X_test = np.array(X_test)
y_test = np.array(y_test)

X_test = X_test / 255
#y_test = to_categorical(y_test, num_classes=4)

In [ ]:
#model = keras.models.load_model('trained_model_1.h5')

results = model.predict(X_test)

results = np.argmax(results,axis = 1)
#y_test = np.argmax(y_test,axis = 1)

In [ ]:
df = pd.DataFrame()
df['results'] = list(results)
df['y_test'] = list(y_test)
df['name'] = list(np.array(data_train.name[-filas_test:]))
df.loc[df['y_test'] == df['results']].count() / df.shape[0]

In [ ]:
fallos = df.loc[(df['y_test'] == 0) & (df['results'] == 3)]

print(fallos.shape)

fallo = fallos.iloc[1,:]

imframe = cv2.imread(dirname + "/" + fallo['name'])
imframe = cv2.cvtColor(imframe, cv2.COLOR_BGR2RGB)
#imframe = cv2.resize(imframe, (s,s))

plt.imshow(imframe)

tipos = [
    'Mascarilla correctamente puesta',
    'Mascarilla por debajo de la nariz',
    'Mascarilla en la barbilla',
    'Sin mascarilla'
]

print('La foto es del tipo: {}, pero la red la clasifico como tipo: {}'.format(tipos[fallo.y_test], tipos[fallo.results]))